In [ ]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using PlotlyJS
using StatsBase, JLD2

# Leverage score approximation

* 1. [MUN graph](#graph)
* 2. [Comparison between exact and JL estimates of leverage scores](#JLestimates) 
    * 2.1 [JL estimates $q=0$](#JLestimatesq0)  
    * 2.2 [JL estimates $q=1$](#JLestimatesq1) 
* 3. [Comparison between exact and empirical estimates of leverage scores](#empLS) 
    * 3.1 [Cycle popping $q=0$ (self-normalized importance sampling)](#empLS_CP_q0)
    * 3.2 [Cycle popping $q=1$ (self-normalized importance sampling)](#empLS_CP_q1) 
    * 3.3 [HKPV $q=0$](#empLS_HKPV_q0) 
    * 3.4 [HKPV $q=1$](#empLS_HKPV_q1) 

<a name="graph"></a>

#### 1. MUN graph  <a id="graph"></a>

In [ ]:
# sample a MUN graph
n = 500
p = 0.2
eta = 0.1
cst = 40

rng = Random.default_rng()
type = "MUN"
meta_g = gen_graph_mun(rng, n, p, eta)
B = magnetic_incidence_matrix(meta_g);

#### 2. Comparison between exact and JL estimates of leverage scores <a id="JLestimates"></a>

2.1 JL estimates for $q=0$ <a id="JLestimatesq0"></a>

In [ ]:
q = 0
lev = leverage_score(B, q)
JL_lev = JL_lev_score_estimates(B, q; cst);

name = "JL-LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)
file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"JL_lev", JL_lev)

println("times saved in "*file)
rel_diff = (lev-JL_lev)./lev;
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))

In [ ]:
file = "figures/data/JL-LSn500p0.2eta0.1q0_data.jld2"
D = load(file)
lev = D["lev"]
JL_lev = D["JL_lev"]

rel_diff = (lev-JL_lev)./lev;

trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)


layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)

# saving
plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")

2.2 JL estimates for $q=1$ <a id="JLestimatesq1"></a>

In [ ]:
q = 1
lev = leverage_score(B, q)
JL_lev = JL_lev_score_estimates(B, q; cst);

rel_diff = (lev-JL_lev)./lev;
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))

name = "JL-LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"JL_lev", JL_lev )

println("times saved in "*file)
rel_diff = (lev-JL_lev)./lev;
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))


In [ ]:
file = "figures/data/JL-LSn500p0.2eta0.1q1_data.jld2"
D = load(file)
lev = D["lev"]
JL_lev = D["JL_lev"]

rel_diff = (lev-JL_lev)./lev;

trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)

name = "JL-LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")

#### 3. Comparison between exact  and empirical estimates of leverage scores <a id="empLS"></a>

3.1 Cycle popping for $q=0$ <a id="empLS_CP_q0"></a>

In [ ]:
nb_samples = Int(1e5)

q = 0.

lev = leverage_score(B, q)
emp_lev = emp_leverage_score(rng, meta_g, q, nb_samples)

rel_diff = vec((lev-emp_lev)./lev);
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))

name = "emp-LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"emp_lev", emp_lev )
println("times saved in "*file)


In [ ]:
D = load(file)
lev = D["lev"]
emp_lev = D["emp_lev"]

rel_diff = vec((lev-emp_lev)./lev);


trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)


plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")

3.2 Cycle popping for $q=1$ <a id="empLS_CP_q1"></a>

In [ ]:
nb_samples = Int(1e5)

q = 1
name = "emp-LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)*"nb_samples"*string(nb_samples)

lev = leverage_score(B, q)
emp_lev = emp_leverage_score(rng, meta_g, q, nb_samples)

rel_diff = vec((lev-emp_lev)./lev);
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))


file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"emp_lev", emp_lev )
println("times saved in "*file)


In [ ]:
file = "figures/data/"*name*"_data.jld2"
D = load(file)
lev = D["lev"]
emp_lev = D["emp_lev"]
rel_diff = vec((lev-emp_lev)./lev);


trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)

name = "emp_LS"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")

3.3 HKPV for $q=0$ <a id="empLS_HKPV_q0"></a>

In [ ]:
nb_samples = Int(100)

q = 0.

lev = leverage_score(B, q)
hkpv = true
emp_lev = emp_leverage_score(rng, meta_g, q, nb_samples;hkpv)

rel_diff = vec((lev-emp_lev)./lev);
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))

name = "emp_LS_HKPV"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"emp_lev", emp_lev )
println("times saved in "*file)


In [ ]:
file = "figures/data/"*name*"_data.jld2"
D = load(file)
lev = D["lev"]
emp_lev = D["emp_lev"]
rel_diff = vec((lev-emp_lev)./lev);


trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)

name = "emp_LS_HKPV"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")

3.4 HKPV for $q=1$ <a id="empLS_HKPV_q1"></a>

In [ ]:
nb_samples = Int(100)

q = 1

lev = leverage_score(B, q)
hkpv = true
emp_lev = emp_leverage_score(rng, meta_g, q, nb_samples;hkpv)

rel_diff = vec((lev-emp_lev)./lev);
println("mean: ", mean(rel_diff), " std: ", std(rel_diff))

name = "emp_LS_HKPV"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

file = "figures/data/"*name*"_data.jld2"
save(file, "lev", lev,"emp_lev", emp_lev )
println("times saved in "*file)


In [ ]:
file = "figures/data/"*name*"_data.jld2"
D = load(file)
lev = D["lev"]
emp_lev = D["emp_lev"]
rel_diff = vec((lev-emp_lev)./lev);

trace = box(y=rel_diff,
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
PlotlyJS.plot(trace,layout)

name = "emp_LS_HKPV"*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

plt = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt,"figures/"*name*".pdf")